In [20]:
from binance.client import Client
import configparser
from binance.websockets import BinanceSocketManager
from twisted.internet import reactor
import pandas as pd 

In [2]:
# Loading keys from config file
config = configparser.ConfigParser()
config.read_file(open('/home/venom/GitHub/cheatsheets/medium/algo_trading/binance_API/secret.cfg'))
api_key = config.get('BINANCE', 'API_KEY')
secret_key = config.get('BINANCE', 'SECRET_KEY')

In [3]:
client = Client(api_key, secret_key)

In [4]:
client.API_URL = 'https://testnet.binance.vision/api'

In [5]:
# get balances for all assets & some account information
print(client.get_account())

{'makerCommission': 0, 'takerCommission': 0, 'buyerCommission': 0, 'sellerCommission': 0, 'canTrade': True, 'canWithdraw': False, 'canDeposit': False, 'updateTime': 1613863611579, 'accountType': 'SPOT', 'balances': [{'asset': 'BNB', 'free': '1000.00000000', 'locked': '0.00000000'}, {'asset': 'BTC', 'free': '1.00000000', 'locked': '0.00000000'}, {'asset': 'BUSD', 'free': '10000.00000000', 'locked': '0.00000000'}, {'asset': 'ETH', 'free': '100.00000000', 'locked': '0.00000000'}, {'asset': 'LTC', 'free': '500.00000000', 'locked': '0.00000000'}, {'asset': 'TRX', 'free': '500000.00000000', 'locked': '0.00000000'}, {'asset': 'USDT', 'free': '10000.00000000', 'locked': '0.00000000'}, {'asset': 'XRP', 'free': '50000.00000000', 'locked': '0.00000000'}], 'permissions': ['SPOT']}


In [6]:
# get balance for a specific asset only (BTC)
print(client.get_asset_balance(asset='BTC'))

{'asset': 'BTC', 'free': '1.00000000', 'locked': '0.00000000'}


In [7]:
# get balances for futures account
# print(client.futures_account_balance())

In [8]:
# get balances for margin account
# print(client.get_margin_account())

In [9]:
# get latest price from Binance API
btc_price = client.get_symbol_ticker(symbol="BTCUSDT")
# print full output (dictionary)
print(btc_price)

{'symbol': 'BTCUSDT', 'price': '7900.00000000'}


In [10]:
print(btc_price["price"])

7900.00000000


In [11]:
btc_price = {'error':False}

def btc_trade_history(msg):
    """ define how to process incoming WebSocket messages """
    #print("Inside function")
    if msg['e'] != 'error':
        print(msg['c'])
        btc_price['last'] = msg['c']
        btc_price['bid'] = msg['b']
        btc_price['ask'] = msg['a']
    else:
        btc_price['error'] = True

In [12]:
# init and start the WebSocket
bsm = BinanceSocketManager(client)
conn_key = bsm.start_symbol_ticker_socket('BTCUSDT', btc_trade_history)
bsm.start()

55852.78000000
55852.78000000
55852.78000000
55852.81000000
55858.93000000
55858.97000000
55858.97000000
55859.32000000
55881.68000000
55881.69000000
55890.27000000
55896.95000000
55881.69000000
55862.82000000
55871.00000000
55871.01000000
55865.72000000
55858.94000000
55877.60000000


In [13]:
# stop websocket
bsm.stop_socket(conn_key)

# properly terminate WebSocket
reactor.stop()

In [14]:
help(BinanceSocketManager)

Help on class BinanceSocketManager in module binance.websockets:

class BinanceSocketManager(threading.Thread)
 |  BinanceSocketManager(client, user_timeout=1800)
 |  
 |  A class that represents a thread of control.
 |  
 |  This class can be safely subclassed in a limited fashion. There are two ways
 |  to specify the activity: by passing a callable object to the constructor, or
 |  by overriding the run() method in a subclass.
 |  
 |  Method resolution order:
 |      BinanceSocketManager
 |      threading.Thread
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, client, user_timeout=1800)
 |      Initialise the BinanceSocketManager
 |      
 |      :param client: Binance API client
 |      :type client: binance.Client
 |      :param user_timeout: Custom websocket timeout
 |      :type user_timeout: int
 |  
 |  close(self)
 |      Close all connections
 |  
 |  run(self)
 |      Method representing the thread's activity.
 |      
 |      You may override

In [15]:
# valid intervals - 1m, 3m, 5m, 15m, 30m, 1h, 2h, 4h, 6h, 8h, 12h, 1d, 3d, 1w, 1M

# get timestamp of earliest date data is available
timestamp = client._get_earliest_valid_timestamp('BTCUSDT', '1d')
print(timestamp)

1612137600000


In [16]:
# request historical candle (or klines) data
bars = client.get_historical_klines('BTCUSDT', '1d', timestamp, limit=1000)

In [17]:
bars

[[1612137600000,
  '33759.83000000',
  '900000.00000000',
  '9000.00000000',
  '882000.00000000',
  '3.33265000',
  1612223999999,
  '239384.12228596',
  188,
  '2.32051200',
  '204624.71086648',
  '0'],
 [1612224000000,
  '882000.00000000',
  '882000.00000000',
  '602000.00000000',
  '772200.00000000',
  '0.12958300',
  1612310399999,
  '108345.91487200',
  102,
  '0.11721900',
  '99547.52580000',
  '0'],
 [1612310400000,
  '702000.00000000',
  '772200.00000000',
  '200000.00000000',
  '700000.00000000',
  '0.17222200',
  1612396799999,
  '124636.73209448',
  132,
  '0.13973400',
  '105948.46129856',
  '0'],
 [1612396800000,
  '500000.00000000',
  '500000.00000000',
  '6000.00000000',
  '9000.00000000',
  '2.00530700',
  1612483199999,
  '83399.00379210',
  131,
  '1.58434600',
  '74828.87856010',
  '0'],
 [1612483200000,
  '37500.00000000',
  '44236.08000000',
  '9000.00000000',
  '44236.08000000',
  '1.96372900',
  1612569599999,
  '76041.09742634',
  206,
  '1.71525800',
  '66148.6

In [ ]:
# option 1 - save to file using json method
with open('btc_bars.json', 'w') as e:
    json.dump(bars, e)

In [ ]:
# option 2 - save as CSV file using the csv writer library
with open('btc_bars.csv', 'w', newline='') as f:
	wr = csv.writer(f)
    for line in bars:
        wr.writerow(line)

In [ ]:
# option 3 - save as CSV file without using a library. 
with open('btc_bars2.csv', 'w') as d:
    for line in bars:
        d.write(f'{line[0]}, {line[1]}, {line[2]}, {line[3]}, {line[4]}\n')

In [18]:
# delete unwanted data - just keep date, open, high, low, close
for line in bars:
    del line[5:]

In [21]:
# option 4 - create a Pandas DataFrame and export to CSV
btc_df = pd.DataFrame(bars, columns=['date', 'open', 'high', 'low', 'close'])
btc_df.set_index('date', inplace=True)
print(btc_df.head())

                          open             high              low  \
date                                                               
1612137600000   33759.83000000  900000.00000000    9000.00000000   
1612224000000  882000.00000000  882000.00000000  602000.00000000   
1612310400000  702000.00000000  772200.00000000  200000.00000000   
1612396800000  500000.00000000  500000.00000000    6000.00000000   
1612483200000   37500.00000000   44236.08000000    9000.00000000   

                         close  
date                            
1612137600000  882000.00000000  
1612224000000  772200.00000000  
1612310400000  700000.00000000  
1612396800000    9000.00000000  
1612483200000   44236.08000000  


In [22]:
# export DataFrame to csv
btc_df.to_csv('btc_bars3.csv')